# Intake Civis Driver

This project is an intake plugin for Civis databases.
It allows easy ingest of tables from the Civis platform,
and integration with other intake YAML catalogs.

## Direct invocation

You can use this driver directly by creating civis catalogs and sources:

In [1]:
import intake_civis

cat = intake_civis.CivisCatalog(
    "City of Los Angeles - Postgres",
    schema="transportation",
)

In [2]:
list(cat)

['bike_trips']

In [3]:
cat.bike_trips

name: "transportation"."bike_trips"
container: dataframe
plugin: ['civis']
description: Civis table bike_trips from City of Los Angeles - Postgres
direct_access: True
user_parameters: []
metadata: 
args: 
  api_key: None
  civis_kwargs: 
  database: City of Los Angeles - Postgres
  table: "transportation"."bike_trips"

In [4]:
df = cat.bike_trips.read()
df.head()

,trip_id,bike_type,end_datetime,end_station,end_station_name,name_group,optional_kiosk_id_group,start_datetime,start_station,start_station_name,visible_id,distance,duration,est_calories,est_carbon_offset
0,139402893,Electric Bike,2020-03-16 00:31:07,3064,Grand & 7th,Annual Pass,DTLA,2020-03-16 00:28:22,3051,7th & Broadway,19005,0.4,3,18,3.158
1,139403093,Electric Bike,2020-03-16 00:43:26,3040,Wilshire & Witmer,Annual Pass,DTLA,2020-03-16 00:33:01,3064,Grand & 7th,19779,1.5,10,60,10.526
2,139403793,Electric Bike,2020-03-16 01:11:54,3049,2nd & Figueroa,1-Ride,DTLA,2020-03-16 01:06:03,3005,7th & Flower,16342,0.8,5,30,5.263
3,139403794,Electric Bike,2020-03-16 01:12:48,4300,Union & Olympic,Monthly Pass,DTLA,2020-03-16 00:56:32,3006,Olive & 8th,18926,2.4,16,96,16.842
4,139405393,Electric Bike,2020-03-16 02:26:24,3026,Figueroa & Cesar Chavez,Annual Pass,DTLA,2020-03-16 02:12:12,4491,Main & Winston,18984,2.1,14,84,14.737


## Usage with a catalog

We can also use this driver from a standard intake YAML catalog:

In [5]:
import intake
cat = intake.open_catalog("catalog.yml")

In [ ]:
# Reading an entire schema as a subcatalog, then pulling a table from that
df = cat.postgres.bike_trips.read()
df.head()

In [ ]:
# Reading a single table
df = cat.bike_trips.read()
df.head()